<a href="https://colab.research.google.com/github/Shasuf/Recommendation-Systems/blob/main/Movie_Rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
print("\n*** Import Libraries ***")
# hides all warnings
import warnings
warnings.filterwarnings('ignore')
# pandas 
import pandas as pd
# numpy
import numpy as np
# matplotlib 
import matplotlib.pyplot as plt
#%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 5)
# sns
import seaborn as sns
print("Done ...")



*** Import Libraries ***
Done ...


In [ ]:
# import utils stored in g-drive / My Drive  / Collab Notebooks
print("\n*** Import Utils ***")
import sys
from google.colab import drive
drive.mount('/content/gdrive')
# To add a directory with your code into a list of directories 
# which will be searched for packages
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')
import utils
print("Done ...")

##############################################


*** Import Utils ***
Mounted at /content/gdrive
Done ...


In [ ]:
print("\n*** Read Data ***")
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/data/mldb-movies-metadata.csv', low_memory=False)
print("Done ...")



*** Read Data ***
Done ...


low memory ensures entire datatype of column remains same.
typically when we have unknown data.

In [ ]:

##############################################################
# Exploratory Data Analysis
##############################################################

# rows & cols
print("\n*** Rows & Cols ***")
print("Rows",df.shape[0])
print("Cols",df.shape[1])


*** Rows & Cols ***
Rows 45460
Cols 24


In [ ]:
# columns
print("\n*** Column Names ***")
print(df.columns)


*** Column Names ***
Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [ ]:
# info
print("\n*** Structure ***")
print(df.info())


*** Structure ***
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45460 entries, 0 to 45459
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45460 non-null  bool   
 1   belongs_to_collection  4490 non-null   object 
 2   budget                 45460 non-null  int64  
 3   genres                 45460 non-null  object 
 4   homepage               7778 non-null   object 
 5   id                     45460 non-null  int64  
 6   imdb_id                45443 non-null  object 
 7   original_language      45449 non-null  object 
 8   original_title         45460 non-null  object 
 9   overview               44506 non-null  object 
 10  popularity             45460 non-null  float64
 11  poster_path            45077 non-null  object 
 12  production_companies   45460 non-null  object 
 13  production_countries   45460 non-null  object 
 14  release_date           45376 non-nu

In [ ]:
# data types
print("\n*** Data Types ***")
print(df.dtypes)


*** Data Types ***
adult                       bool
belongs_to_collection     object
budget                     int64
genres                    object
homepage                  object
id                         int64
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity               float64
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                    int64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                       bool
vote_average             float64
vote_count                 int64
dtype: object


In [ ]:
# head
#print(df[['genres','title','vote_average','vote_count']].head())
print(df[['title','vote_average','vote_count']].head())

                         title  vote_average  vote_count
0                    Toy Story           7.7        5415
1                      Jumanji           6.9        2413
2             Grumpier Old Men           6.5          92
3            Waiting to Exhale           6.1          34
4  Father of the Bride Part II           5.7         173


In [ ]:

##################################################
# Simple Recommender 
##################################################
# The Simple Recommender offers generalized recommnendations to every user based 
# on movie popularity and (sometimes) genre. The basic idea behind this recommender 
# is that movies that are more popular and more critically acclaimed will have 
# a higher probability of being liked by the average audience. This model does 
# not give personalized recommendations based on the user.
#
# The implementation of this model is extremely trivial. All we have to do is 
# sort our movies based on ratings and popularity and display the top movies of 
# our list. As an added step, we can pass in a genre argument to get the top movies 
# of a particular genre.



# top movies 
# based on weighted average calculated from vote_average & vote_count with 
# 50 percentile cut-off
# wa = ( (v/(v+m)) * R) + ( (m/(m+v)) * C)
# where
# v is the number of votes for the movie
# m is the minimum votes required to be listed in the chart
# R is the average rating of the movie
# C is the mean vote across the whole report / table

In [ ]:
# C is the mean vote across the whole report
print("\n*** Global Vote Average C ***")
C = df['vote_average'].mean()
print(C)


*** Global Vote Average C ***
5.618207215133889


vote average across entire file 

In [ ]:
# m is the minimum votes required to be listed in the chart
print("\n*** Min Votes Required ***")
m = df['vote_count'].quantile(0.50)
print(m)



*** Min Votes Required ***
10.0


minimum 10ratings are required, otherwise it will not be considered

In [ ]:
# Filter out all qualified movies into a new DataFrame
print("\n*** Filter > Min Votes ***")
df = df[df['vote_count'] >= m]
print(type(df))
print(df.shape)
print(df.shape)
print(df.head())


*** Filter > Min Votes ***
<class 'pandas.core.frame.DataFrame'>
(22931, 24)
(22931, 24)
   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt01147


removing weighting at 50(quantile) only 10% have good ratings

#**you want more data for prediction go for less percentile** unless you have suffiecient data

In [ ]:
# function that computes the weighted rating of each movie
def weightedRating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IdfB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

computing the weighted average, calculating the V,R from individual row and then returning.

In [ ]:
# define a new feature 'score' and calculate its value with `weighted_rating()`
print("\n*** Create Score ***")
df['score'] = df.apply(weightedRating, axis=1)
print(type(df))
print(df.shape)
print(df.head())
print(df[['title','score']].head())


*** Create Score ***
<class 'pandas.core.frame.DataFrame'>
(22931, 25)
   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                

In [ ]:
# sort movies based on score calculated above
print("\n*** Sort On Score ***")
df = df.sort_values('score', ascending=False)
print(df[['title','vote_average','vote_count','score']].head())



*** Sort On Score ***
                             title  vote_average  vote_count     score
10309  Dilwale Dulhania Le Jayenge           9.1         661  9.048110
43314              Planet Earth II           9.5          50  8.853035
39079                 Planet Earth           8.8         176  8.628936
314       The Shawshank Redemption           8.5        8358  8.496556
834                  The Godfather           8.5        6024  8.495224


In [ ]:
# print the top 15 movies
print("\n*** Top 5 Movies ***")
print(df[['title', 'vote_count', 'vote_average', 'score']].head(15))


*** Top 5 Movies ***
                             title  vote_count  vote_average     score
10309  Dilwale Dulhania Le Jayenge         661           9.1  9.048110
43314              Planet Earth II          50           9.5  8.853035
39079                 Planet Earth         176           8.8  8.628936
314       The Shawshank Redemption        8358           8.5  8.496556
834                  The Godfather        6024           8.5  8.495224
40245                   Your Name.        1030           8.5  8.472290
43517                       Cosmos          41           9.1  8.417296
33416                        Human          98           8.6  8.323908
7643            Sansho the Bailiff          68           8.7  8.304898
12481              The Dark Knight       12269           8.3  8.297816
2843                    Fight Club        9678           8.3  8.297232
292                   Pulp Fiction        8670           8.3  8.296910
522               Schindler's List        4436         

In [ ]:
# handle genres wise top n
print("\n*** Genre Dict to List ***")
print(df['genres'].head())


*** Genre Dict to List ***
10309    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
43314                  [{'id': 99, 'name': 'Documentary'}]
39079                  [{'id': 99, 'name': 'Documentary'}]
314      [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...
834      [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...
Name: genres, dtype: object


this is a json file we need to extract "name", if its empty use lamba

In [ ]:
# this is dictionary ... convert to list
df['genres'] = df['genres'].fillna('[]').apply(eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
print(df['genres'].head())


10309    [Comedy, Drama, Romance]
43314               [Documentary]
39079               [Documentary]
314                [Drama, Crime]
834                [Drama, Crime]
Name: genres, dtype: object


whatever we want to extract we put inthe above formula
extract only name, if it is NULL fill with NA and apply lamba.

In [ ]:
lGenres = ['Action','Adventure','Animation","Children','Comedy', 'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
print(lGenres)


['Action', 'Adventure', 'Animation","Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [ ]:
# movies of a genre
def getGenreMovies(pGenre):
    global df
    mask = df['genres'].apply(lambda x: pGenre in x)
    #print(mask[0:5])
    mg = df[mask]
    # print the top 5 movies
    print("\nGenre:", pGenre)
    print("==============================================")
    print(mg[['title', 'vote_count', 'vote_average', 'score']].head(5))

In [ ]:
# sample genre
print("\n*** Sample Genre List ***")
getGenreMovies('Comedy')


*** Sample Genre List ***

Genre: Comedy
                             title  vote_count  vote_average     score
10309  Dilwale Dulhania Le Jayenge         661           9.1  9.048110
2211             Life Is Beautiful        3643           8.3  8.292659
351                   Forrest Gump        8147           8.2  8.196835
18465             The Intouchables        5410           8.2  8.195237
3189                   City Lights         444           8.2  8.143132


In [ ]:
# get top 5 for all genres
print("\n*** Top 5 For All Genres ***")
for vGenre in lGenres:
    getGenreMovies(vGenre)


*** Top 5 For All Genres ***

Genre: Action
                         title  vote_count  vote_average     score
12481          The Dark Knight       12269           8.3  8.297816
1154   The Empire Strikes Back        5998           8.2  8.195703
1910             Seven Samurai         892           8.2  8.171377
43184         Band of Brothers         725           8.2  8.164874
15480                Inception       14075           8.1  8.098238

Genre: Adventure
                         title  vote_count  vote_average     score
5481             Spirited Away        3968           8.3  8.293258
1154   The Empire Strikes Back        5998           8.2  8.195703
9698      Howl's Moving Castle        2049           8.2  8.187461
2884         Princess Mononoke        2041           8.2  8.187412
15480                Inception       14075           8.1  8.098238

Genre: Animation","Children
Empty DataFrame
Columns: [title, vote_count, vote_average, score]
Index: []

Genre: Comedy
             

In [ ]:
# get top 5 based on genre of movie
print("\n*** Get Genre List Based On Current Movie ***")
vCrnntMovie = "The Godfather"
print(vCrnntMovie)
dfs = df[ df['title']==vCrnntMovie ]
print(dfs)
lGenres = dfs['genres']
print(lGenres)
try:
    lGenres = dfs.iloc[0,3]
    print(lGenres)
    # get top 5 for all genres
    for vGenre in lGenres:
        print("\n" + vGenre)
        getGenreMovies(vGenre)
except:
    print("Title Not Found ...")


*** Get Genre List Based On Current Movie ***
The Godfather
     adult                              belongs_to_collection   budget  \
834  False  {'id': 230, 'name': 'The Godfather Collection'...  6000000   

             genres                      homepage   id    imdb_id  \
834  [Drama, Crime]  http://www.thegodfather.com/  238  tt0068646   

    original_language original_title  \
834                en  The Godfather   

                                              overview  ...    revenue  \
834  Spanning the years 1945 to 1955, a chronicle o...  ...  245066411   

    runtime                                   spoken_languages    status  \
834   175.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...  Released   

                        tagline          title  video vote_average vote_count  \
834  An offer you can't refuse.  The Godfather  False          8.5       6024   

        score  
834  8.495224  

[1 rows x 25 columns]
834    [Drama, Crime]
Name: genres, dtype: object
